In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import os
import re

# Path ke ChromeDriver kamu
chrome_driver_path = 'E:\Liga 1 Fix\chromedriver-win64\chromedriver.exe'
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)
driver.maximize_window()

link_klub = pd.read_csv('Link_Klub.csv')
link_klub = link_klub['Link Klub'].tolist()

for url in link_klub[:2]:
    # Buka halaman Persib Bandung
    driver.get(url)

    # Ambil nama tim dari URL
    match = re.search(r'/tim/sepak-bola/([^/]+)/', url)
    if match:
        nama_tim = match.group(1).replace('-', ' ')  # Ganti dash dengan underscore kalau mau
    else:
        nama_tim = "tim_tidak_dikenal"

    nama_tim = nama_tim.title()
    print(nama_tim)

    time.sleep(5)

    data = []
    index = 1
    driver.execute_script("window.scrollBy(0, 500);")

    # Klik tombol dropdown
    dropdown_button = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/div[2]/div[1]/div[2]/button')
    dropdown_button.click()
    print("Klik 1")
    time.sleep(1)

    # Pilih opsi kedua (misal: statistik pemain)
    opsi_kedua = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/div[2]/div[1]/div[2]/div/div/div/ul/li[2]')
    opsi_kedua.click()
    print("Klik 2")
    time.sleep(2)

    # Klik tombol "Show All"
    show_all_btn = driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/div[3]/div/button')
    show_all_btn.click()
    print("Klik 3")
    time.sleep(2)

    # Scroll 3 kali
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

    while True:
        # XPath Nama Pemain
        name_xpath = f"/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/a[{index}]/div/div/span"
        # XPath Rating (pseudo-element ::after)
        rating_xpath = f"/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/a[{index}]/div/span[2]/div/span/div"
        link_xpath = f"/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/a[{index}]"
        posisi_xpath = f"/html/body/div[1]/main/div[2]/div/div[2]/div[2]/div[4]/a[{index}]/div/div/div/span"

        try:

            # Ambil nama pemain
            name_element = driver.find_element(By.XPATH, name_xpath)
            name = name_element.text.strip()
            posisi_element = driver.find_element(By.XPATH, posisi_xpath)
            posisi = posisi_element.text.strip()
            link_element = driver.find_element(By.XPATH, link_xpath)
            link = link_element.get_attribute("href")

        except NoSuchElementException:
            break  # Keluar loop kalau nama nggak ditemukan

        # Ambil rating via pseudo-element
        js_script = """
            var element = document.evaluate(arguments[0], document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            if (element) {
                var style = window.getComputedStyle(element, '::after');
                return style.getPropertyValue('content');
            } else {
                return null;
            }
        """
        rating_content = driver.execute_script(js_script, rating_xpath)
        rating = rating_content.strip('"') if rating_content and rating_content != 'none' else None

        # Simpan ke list
        data.append({
            "Name": name,
            "Rating": rating,
            "Posisi": posisi,
            "Link": link
        })

        index += 1

    # Convert ke DataFrame dan tampilkan
    df = pd.DataFrame(data)

    output_dir = os.path.join('sementara-rating')
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f'{nama_tim}-rating.csv')
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f'Sukses menyimpan data pemain dari {nama_tim}')

driver.quit()


Persib Bandung
Klik 1
Klik 2
Klik 3
Sukses menyimpan data pemain dari Persib Bandung
Dewa United Fc
Klik 1
Klik 2
Klik 3
Sukses menyimpan data pemain dari Dewa United Fc
